# Sentiment Analysis

In the blocks of code below, we conduct sentiment analysis on Reddit data.

In [1]:
import praw

# Authenticate

user_agent = "Reddit_Scrapper 1.0 by /u//Sure-Drama5780"
reddit = praw.Reddit(
    client_id = "Bjk_uxsBZlpdVwKTNZlzxQ",
    client_secret = "LaZgrTga58GqmrqtmKoWmohvCSugdw",
    user_agent = user_agent
)

In [2]:
import pandas as pd
import numpy as np
import re

headlines = set()

# Hot topics

for submission in reddit.subreddit("news").hot(limit = None):
    print(submission.title)
    print(submission.id)
    print(submission.author)
    print(submission.created_utc)
    print(submission.score)
    print(submission.upvote_ratio)
    print(submission.url)
    break
    headlines.add(submission.title)
print(len(headlines))

Three Palestinian students shot and wounded in Burlington, Vermont, police say
184got7
sleepy_time_Ty
1701021989.0
8442
0.91
https://theguardian.com/us-news/2023/nov/26/palestinian-students-shot-wounded-vermont
0


In [3]:
# Number of people talking about the topic

for submission in reddit.subreddit("news").hot(limit = None):
    headlines.add(submission.title)
print(len(headlines))

224


In [4]:
reddit_df = pd.DataFrame(headlines)
reddit_df.head()

,0
0,6 Colorado officers charged with failing to in...
1,Former San Jose State director of sports medic...
2,Richest 1% account for more carbon emissions t...
3,Fire at Freemason Lodge in Mill Bay considered...
4,Israel drops leaflets warning people to flee s...


In [5]:
reddit_df.columns = ["Titles"]
reddit_df.head()

,Titles
0,6 Colorado officers charged with failing to in...
1,Former San Jose State director of sports medic...
2,Richest 1% account for more carbon emissions t...
3,Fire at Freemason Lodge in Mill Bay considered...
4,Israel drops leaflets warning people to flee s...


In [6]:
reddit_df.Titles.duplicated().sum()

0

In [7]:
# Clean the reddit posts
def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0–9]+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r'https?:\/\/\S+', '', text)
    text = re.sub(r':', '', text)
    return text

reddit_df['Titles'] = reddit_df["Titles"].apply(cleanTxt)

reddit_df.head()

,Titles
0,6 Colorado officers charged with failing to in...
1,Former San Jose State director of sports medic...
2,Richest 1% account for more carbon emissions t...
3,Fire at Freemason Lodge in Mill Bay considered...
4,Israel drops leaflets warning people to flee s...


In [8]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F1E0-\U0001F1FF"
                               u"\U00002500-\U00002BEF"
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

reddit_df["Titles"] = reddit_df["Titles"].apply(remove_emoji)
reddit_df.head()

,Titles
0,6 Colorado officers charged with failing to in...
1,Former San Jose State director of sports medic...
2,Richest 1% account for more carbon emissions t...
3,Fire at Freemason Lodge in Mill Bay considered...
4,Israel drops leaflets warning people to flee s...


In [9]:
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS

def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

reddit_df['Subjectivity'] = reddit_df['Titles'].apply(getSubjectivity)
reddit_df['Polarity'] = reddit_df['Titles'].apply(getPolarity)

reddit_df.head()

,Titles,Subjectivity,Polarity
0,6 Colorado officers charged with failing to in...,0.000000,0.00
1,Former San Jose State director of sports medic...,0.416667,0.25
2,Richest 1% account for more carbon emissions t...,0.500000,0.50
3,Fire at Freemason Lodge in Mill Bay considered...,0.000000,0.00
4,Israel drops leaflets warning people to flee s...,0.000000,0.00


In [10]:
def getInsight(score):
    if score < 0:
        return "Negative"
    elif score == 0:
        return "Neutral"
    else:
        return "Positive"

reddit_df['Insight'] = reddit_df["Polarity"].apply(getInsight)
reddit_df.head()

,Titles,Subjectivity,Polarity,Insight
0,6 Colorado officers charged with failing to in...,0.000000,0.00,Neutral
1,Former San Jose State director of sports medic...,0.416667,0.25,Positive
2,Richest 1% account for more carbon emissions t...,0.500000,0.50,Positive
3,Fire at Freemason Lodge in Mill Bay considered...,0.000000,0.00,Neutral
4,Israel drops leaflets warning people to flee s...,0.000000,0.00,Neutral


# The next blocks of code fetches Options data from Yahoo Finance

In [41]:
import yfinance as yf
import datetime

def options_chain(symbol):
    tk = yf.Ticker(symbol)
    
    expiration_dates = tk.options
    
    # Get options for each expiration
    options = pd.DataFrame()
    
    for e in expiration_dates:
        opt = tk.option_chain(e)
        #opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt = pd.concat([opt.calls, opt.puts], axis = 0)
        opt[['expirationDate']] = e
        #options = options.append(opt, ignore_index = True)
        options = pd.concat([options, opt], axis = 0)
    
    options['expirationDate'] = pd.to_datetime(options['expirationDate']) + datetime.timedelta(days = 1)
    options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days / 365
    
    options['CALL'] = options['contractSymbol'].str[4:].apply(lambda x: "C" in x)
    
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['mark'] = (options['bid'] + options['ask'])
    
    return options

options_chain("META")
#tk = yf.Ticker("META")

#exps = tk.options

#opt = tk.option_chain(exps[0])

#opt = pd.concat([opt.calls, opt.puts], axis=0)
#opt['expirationDate'] = exps[0]
#opt

KeyError: 'expirationDate'